In [70]:
import uproot, glob

NameError: name 'globalMuon' is not defined

In [2]:
f = uproot.open("./VBF_HToInvisible_M125_14TeV_pythia8_PU200_V7_4_2.root")

In [60]:
t = f['l1PhaseIITree/L1PhaseIITree']
t.allkeys()

#t.arrays(['pfCandId', 'pfCandEt'])

[b'L1PhaseII',
 b'z0Puppi',
 b'z0VertexTDR',
 b'nVertices',
 b'z0Vertices',
 b'nL1TkPVs',
 b'z0L1TkPV',
 b'sumL1TkPV',
 b'nTaus',
 b'tauEt',
 b'tauEta',
 b'tauPhi',
 b'tauIEt',
 b'tauIEta',
 b'tauIPhi',
 b'tauIso',
 b'tauBx',
 b'tauTowerIPhi',
 b'tauTowerIEta',
 b'tauRawEt',
 b'tauIsoEt',
 b'tauNTT',
 b'tauHasEM',
 b'tauIsMerged',
 b'tauHwQual',
 b'nJets',
 b'jetEt',
 b'jetEta',
 b'jetPhi',
 b'jetIEt',
 b'jetIEta',
 b'jetIPhi',
 b'jetBx',
 b'jetTowerIPhi',
 b'jetTowerIEta',
 b'jetRawEt',
 b'jetSeedEt',
 b'jetPUEt',
 b'jetPUDonutEt0',
 b'jetPUDonutEt1',
 b'jetPUDonutEt2',
 b'jetPUDonutEt3',
 b'nCaloJets',
 b'caloJetEt',
 b'caloJetEta',
 b'caloJetPhi',
 b'caloJetBx',
 b'nGlobalMuons',
 b'globalMuonPt',
 b'globalMuonEta',
 b'globalMuonPhi',
 b'globalMuonEtaAtVtx',
 b'globalMuonPhiAtVtx',
 b'globalMuonIEt',
 b'globalMuonIEta',
 b'globalMuonIPhi',
 b'globalMuonIEtaAtVtx',
 b'globalMuonIPhiAtVtx',
 b'globalMuonIDEta',
 b'globalMuonIDPhi',
 b'globalMuonChg',
 b'globalMuonIso',
 b'globalMuonQu

In [4]:
pt = t.array(b"pfCandEt").pad(10).fillna(0)
eta = t.array(b"pfCandEta").pad(10).fillna(0)
phi = t.array(b"pfCandPhi").pad(10).fillna(0)
ipt = pt[:, :10]
ieta = eta[:, :10]
iphi = phi[:, :10]
print(ipt, ieta, iphi)

[[5.25 5.5 4.0 ... 2.25 2.25 1.25] [3.25 2.25 2.0 ... 4.75 4.0 3.5] [16.0 4.25 4.25 ... 5.75 10.75 4.5] ... [116.0 4.5 1.25 ... 2.0 2.25 16.75] [9.25 2.75 6.5 ... 2.0 2.0 12.0] [1.75 1.25 1.25 ... 2.75 2.5 2.5]] [[-0.13089969754219055 -1.4718300104141235 -1.3731600046157837 ... -1.2062499523162842 -1.0385199785232544 -1.1999138593673706] [0.13089969754219055 0.13089969754219055 -0.4101523756980896 ... -0.42497000098228455 -1.4390100240707397 -1.4850399494171143] [-1.4748032093048096 -1.3700834512710571 -1.148900032043457 ... -1.3962633609771729 -1.405750036239624 0.022789999842643738] ... [0.811578094959259 0.7766715288162231 0.811578094959259 ... 0.2452400028705597 -2.0445899963378906 -2.2296581268310547] [-0.008726646192371845 -1.2130038738250732 -1.1539599895477295 ... -0.11178000271320343 1.3176300525665283 -1.1082841157913208] [-0.8988445401191711 -0.4974188208580017 0.9337511658668518 ... 1.4227700233459473 1.023169994354248 1.1838300228118896]] [[-0.5323253870010376 0.3240549862

In [27]:
from torch.utils.data import Dataset, DataLoader

In [71]:
class P2L1NTP(Dataset):
    def __init__(self, dir_name, feature_name = 'Particles', 
                 label_name = 'Labels', tree_name="l1PhaseIITree/L1PhaseIITree",
                 sequence_length=50, verbose=False):
        self.tree_name = tree_name
        self.feature_name = feature_name
        self.label_name = label_name
        self.sequence_length = sequence_length
        self.file_names = glob.glob(dir_name+'/*.root')
        self.totalEny = 0
        self.PhysicsObt = {
            "jet" : 12,
            "tau" : 12,
            "EG" : 12,
            "globalMuon": 12
        }
        self.SelBranches={}
        for var in ["Et", "Eta", "Phi"]:
            for obj in self.PhysicsObt.keys():
                if "Muon" in obj and var == "Et":
                    var = "Pt"
                self.SelBranches[obj+var] = self.PhysicsObt[obj]
        #self.SelBranches=["pfCandEt", "pfCandEta", "pfCandPhi"]
        self.cache = uproot.cache.ArrayCache(1024**2)
        self.upTree = uproot.iterate(dir_name+'/*.root',  self.tree_name, cache=self.cache, branches=self.SelBranches.keys(), namedecode="utf8", reportentries=True, entrysteps=self.sequence_length)
        self.startIdx = -1
        self.endIdx = -1
        
    
    def __len__(self):
        for f in self.file_names:
            temptree = uproot.open(f)[self.tree_name]
            #print(temptree.pandas.df(branches=self.SelBranches))
            self.totalEny += temptree.numentries
        return self.totalEny
    
    def __getitem__(self, idx):
        while idx < self.startIdx or idx > self.endIdx:
            self.startIdx, self.endIdx, self.events =  next(self.upTree)
        #print(self.startIdx, self.endIdx, idx)
        locidx = idx-self.startIdx
        reflatnp = []
        for b, ln in self.SelBranches.items():
            g  = self.events[b].pad(ln).fillna(0)[locidx, :ln]
            reflatnp.append(g)
            #print(b)
        return reflatnp
b = P2L1NTP("./")
#print(len(b))
import numpy as np
gg = np.vstack(b[134]).reshape((-1,),order='F')
#print(gg, gg.shape)

[ 2.28000001e+02  2.60000010e+01  5.35042810e+00  5.00000000e+00
  3.56699991e+00 -4.35000002e-01 -5.15070021e-01 -2.26200000e+00
  1.30499989e-01 -7.58685414e-01 -7.60214210e-01 -2.85807331e+00
  2.22500001e+02  2.50000010e+01  5.51762915e+00  2.50000000e+00
  4.26300001e+00  6.95999980e-01 -3.05550039e-01 -1.80525000e+00
 -7.58685414e-01 -2.84668547e+00  1.59597838e+00  3.06514800e+00
  2.04000001e+02  2.50000010e+01  1.07008562e+01  0.00000000e+00
  3.91499996e+00  6.95999980e-01  5.15070021e-01  0.00000000e+00
  1.87049997e+00  2.39249992e+00  1.99715957e-01  0.00000000e+00
  1.85500001e+02  2.50000010e+01  1.58865492e+01  0.00000000e+00
 -4.61100006e+00 -2.60999978e-01 -1.72984767e+00  0.00000000e+00
 -4.10685364e-01  1.39199996e+00 -1.54708672e+00  0.00000000e+00
  1.82000001e+02  2.45000010e+01  0.00000000e+00  0.00000000e+00
  4.78499985e+00  8.69999975e-02  0.00000000e+00  0.00000000e+00
  2.56649995e+00 -2.02018530e+00  0.00000000e+00  0.00000000e+00
  1.78500001e+02  2.40000

In [78]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

dataloader = DataLoader(b, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder()
#model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        print(data)
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        #img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))

torch.save(model.state_dict(), './sim_autoencoder.pth')